In [1]:
from functools import wraps
import mmh3

In [2]:
# node with their sockets
from server_config import NODES

class HRW():
    # creator of ring
    def __init__(self, nodes):
        assert len(nodes) > 0
        self.nodes = nodes

    # get node's socket by index
    def get_node(self, key_hex):
        # the md5 hash was digest by hex, so convert to int base on 16
        key = int(key_hex, 16)
        # get the node index by modulus
        node_index = key % len(self.nodes)
        # return the socket of the node
        return self.nodes[node_index]

    def hrw(self,key_hex):
        weights = [mmh3.hash(key_hex,i) for i in range(len(NODES))]
        node_index = weights.index(max(weights))
        return self.nodes[node_index]

In [3]:
def test():
    # create a ring with provided Nodes sockets
    ring = HRW(nodes=NODES)
    # get the node socket of provided md5 hash value
    node = ring.get_node('9ad5794ec94345c4873c4e591788743a')
    print(node)
    print(ring.get_node('ed9440c442632621b608521b3f2650b8'))
    node2 = ring.hrw('9ad5794ec94345c4873c4e591788743a')
    print(node2)
    print(ring.hrw('ed9440c442632621b608521b3f2650b8'))

In [4]:
test()

{'host': '127.0.0.1', 'port': 4002}
{'host': '127.0.0.1', 'port': 4000}
{'host': '127.0.0.1', 'port': 4003}
{'host': '127.0.0.1', 'port': 4002}


In [5]:
import bisect
class CH():
    # init the consistent node ring
    def __init__(self, nodes,replication_factor = 8):
        assert len(nodes) > 0
        self.nodes = nodes
        # maximum slot numbers
        self.M = pow(2,32)
        # number of replica vnode
        self.rep = replication_factor
        # record the location of node as a hash code on the ring
        self.nodering = []
        # record the hash-node mapping
        self.nodehash = {}
        # initiate the ring
        for node in self.nodes:
            self.add_node(node)
    # add a node to the ring
    def add_node(self,node):
        # first get the hash by node name
        _hash = mmh3.hash(str(node).encode()) % self.M
        # add hash-node mapping to dict
        self.nodehash[_hash] = node
        # add ndoe to the ring
        self.nodering.append(_hash)
        # genereate virtual nodes
        for i in range(self.rep):
            # generate hash code of vnode by name
            v_hash = mmh3.hash((str(node)+f"#{i}").encode()) % self.M
            self.nodehash[v_hash] = node
            self.nodering.append(v_hash)
        # sort the ring
        self.nodering.sort()
    # remove a node
    def remove_node(self,node):
        # store node and relative vnode in a remove list
        rmlist = []
        # get hash of the node and vnode
        _hash = mmh3.hash(str(node).encode()) % self.M
        rmlist.append(nodering.append(_hash))
        for i in range(self.rep):
            v_hash = mmh3.hash((str(node)+f"#{i}").encode()) % self.M
            rmlist.append(v_hash)
        #find these node and remove them from ring and dict
        for each in rmlist:
            self.nodering.remove(each)
            self.nodehash.pop(each)
    # get the node index of a provided key, as well as the next node for data replication
    def get_node(self, key):
        # get key hash
        k_hash = mmh3.hash(key) % self.M
        # find the node by binary search
        n_i = bisect.bisect_left(self.nodering,k_hash) % len(self.nodering)
        node_list = []
        node_list.append(self.nodehash[self.nodering[n_i]])
        n_nxt = (n_i + 1) % len(self.nodering)
        # avoid same data replication in same actual node
        while self.nodehash[self.nodering[n_i]] == self.nodehash[self.nodering[n_nxt]]:
            n_nxt = (n_nxt + 1) % len(self.nodering)
        node_list.append(self.nodehash[self.nodering[n_nxt]])
        return node_list
    # pring node ring and dict
    def check(self):
        print(self.nodering)
        print(self.nodehash)

In [6]:
import string
import random
def randomString(stringLength=8):
    letters = string.ascii_lowercase
    return ''.join(random.choice(letters) for i in range(stringLength))

In [7]:
from sample_data import USERS
from pickle_hash import serialize_PUT
import string
import random

def randomString(stringLength=8):
    letters = string.ascii_lowercase
    return ''.join(random.choice(letters) for i in range(stringLength))

#generate test set
test = []
for i in range(1000):
    #generate random object
    name = randomString()
    user = {'name':name,"email":name + "@email.com","age":random.randint(0,100)}
    data_bytes, key = serialize_PUT(user)
    # first find out the nodes by the key, then send the data by that client
    test.append(key)

CH_ring = CH(nodes=NODES)
HRW_ring = HRW(nodes=NODES)

In [8]:
def test_HRW(R,keys):
    # create a ring with provided Nodes sockets
    ring = R
    # calculate distribution
    dist = [0]*len(NODES)
    # get the node socket of provided md5 hash value
    for key in keys:
        node = ring.hrw(key)['port']%len(dist)
        dist[node] += 1
    print(dist)
test_HRW(HRW_ring,test)

[243, 267, 235, 255]


In [9]:
def test_CH(R,keys):
    # create a ring with provided Nodes sockets
    ring = R
    # calculate distribution
    dist = [0]*len(NODES)
    # get the node socket of provided md5 hash value
    for key in keys:
        node = ring.get_node(key)[0]['port']%len(dist)
        dist[node] += 1
    print(dist)
test_CH(CH_ring,test)

[249, 377, 234, 140]
